In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Fan-T5-XXL

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_flan_t5_xxl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_flan_t5_xxl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_flan_t5_xxl.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
    (a Python-3 GPU notebook with preinstalled HuggingFace/transformer libraries is recommended)
  </td>
</table>

## Overview

This notebook demonstrates finetuning [google/flan-t5-xxl](https://huggingface.co/google/flan-t5-xxl) with [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset and deploying it on Vertex AI for online prediction.

### Objective

- Finetune the flan-t5-xxl model with [Vertex AI custom training](https://cloud.google.com/vertex-ai/docs/training/overview).
- Upload the model to [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model to a [Vertex AI Endpoint resource](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for text summarization task.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Setup environment

**NOTE**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Colab only

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for deploying fine tuned model to Vertex AI endpoint.

Fill following variables for experiments environment:

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Define constants

In [ ]:
# The pre-built training docker image. It contains training scripts and models.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-flan-t5-train"

# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-transformers-serve"

### Define common functions

In [ ]:
import os
from datetime import datetime


# Create a job name string with a prefix.
def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


# Deploy a model to Vertex AI endpoint.
def deploy_model(model_id, task):
    model_name = "flan-t5-xxl"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="a2-highgpu-1g",
        accelerator_type="NVIDIA_TESLA_A100",
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint

## Fine tune model

This section fine tunes the Flan-t5-XXL model with [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset for the text summarization task.

One `a2-highgpu-8g` machine with 8 `NVIDIA_TESLA_A100` is required to run the fine-tuning job. The fine-tuning job takes about 11.5 hours to complete.

The fine-tuned model will be saved after the job finishs and it can then be loaded for inference.

In [ ]:
machine_type = "a2-highgpu-8g"
gpu_type = "NVIDIA_TESLA_A100"
num_gpus = 8

job_name = create_job_name("flan-t5-xxl")
output_dir = f"/gcs/{GCS_BUCKET}/flan-t5-xxl"

job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name, container_uri=TRAIN_DOCKER_URI, command=["torchrun"]
)

job.run(
    args=[
        "--nnodes=1",
        "--node_rank=0",
        f"--nproc-per-node={num_gpus}",
        "train.py",
        f"--output_dir={output_dir}",
        "--bf16=True",
        "--model_id=google/flan-t5-xxl",
        "--data_path=cnn_dailymail",
        "--data_name=3.0.0",
        "--gradient_checkpointing=True",
        "--train_epochs=3",
        "--train_batch_size=8",
        "--eval_batch_size=8",
        "--lr=1e-4",
        "--checkpoint_strategy=no",
    ],
    boot_disk_size_gb=600,
    replica_count=1,
    machine_type=machine_type,
    accelerator_type=gpu_type,
    accelerator_count=num_gpus,
)

## Upload and Deploy models

This section uploads the fine-tuned model to Model Registry and deploys it on the Endpoint.

One `a2-highgpu-1g` machine with 1 `NVIDIA_TESLA_A100` is required to deploy Flan-t5-XXL model.

The model deployment step will take ~20 minutes to complete.

In [ ]:
# Set the model_id to "google/flan-t5-xxl" to load the OSS pre-trained model.
model, endpoint = deploy_model(
    model_id=f"gs://{GCS_BUCKET}/flan-t5-xxl/release", task="summarization"
)

NOTE: The model weights will be downloaded after the deployment succeeds. Thus additional 5 minutes of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

In [ ]:
text = "LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in 'Harry Potter and the Order of the Phoenix' To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. 'I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar,' he told an Australian interviewer earlier this month. 'I don't think I'll be particularly extravagant. 'The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs.' At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film 'Hostel: Part II,' currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. 'I'll definitely have some sort of party,' he said in an interview. 'Hopefully none of you will be reading about it.' Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. 'People are always looking to say 'kid star goes off the rails,'' he told reporters last month. 'But I try very hard not to go that way because it would be too easy for them.' His latest outing as the boy wizard in 'Harry Potter and the Order of the Phoenix' is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called 'My Boy Jack,' about author Rudyard Kipling and his son, due for release later this year. He will also appear in 'December Boys,' an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's 'Equus.' Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: 'I just think I'm going to be more sort of fair game,' he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed."
instances = [
    {
        "text": text,
    }
]
response = endpoint.predict(instances=instances).predictions
print(response)

Clean up resources:

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()